# Building a `Memgraph` database with the `MetaKG` index

In [12]:
import pprint as pp 
import mgclient # memgraph client

from controller.metakg import MetaKG
from model import ConsolidatedMetaKGDoc

In [13]:
conn = mgclient.connect(host="localhost", port=7687)   # connect to memgraph
conn.autocommit = True  # autocommit mode is required for DDL queries
cursor = conn.cursor() # cursor is used to execute queries

In [3]:
# # load all 
cursor.execute("CALL mg.load_all();")


In [15]:
# Execute the query to list all procedures
cursor.execute("CALL mg.procedures() YIELD *")

# Fetch and display the results
procedures = cursor.fetchall()
for proc in procedures:
    print(proc)

(True, False, 'algo.all_simple_paths', '/usr/lib/memgraph/query_modules/algo.so', 'algo.all_simple_paths(start_node :: NODE, end_node :: NODE, relationship_types :: LIST OF STRING, max_length :: INTEGER) :: (path :: PATH)')
(True, False, 'algo.astar', '/usr/lib/memgraph/query_modules/algo.so', 'algo.astar(start :: NODE, target :: NODE, config :: MAP) :: (path :: PATH, weight :: FLOAT)')
(True, False, 'algo.cover', '/usr/lib/memgraph/query_modules/algo.so', 'algo.cover(nodes :: LIST OF NODE) :: (rel :: RELATIONSHIP)')
(True, False, 'betweenness_centrality.get', '/usr/lib/memgraph/query_modules/betweenness_centrality.so', 'betweenness_centrality.get(directed = true :: BOOLEAN, normalized = true :: BOOLEAN, threads = 8 :: INTEGER) :: (betweenness_centrality :: FLOAT, node :: NODE)')
(True, False, 'betweenness_centrality_online.get', '/usr/lib/memgraph/query_modules/betweenness_centrality_online.so', 'betweenness_centrality_online.get(normalize = true :: BOOLEAN) :: (betweenness_centrality

In [3]:
# Load only the specific module
cursor.execute("CALL mg.load('algo');")

Find a node pair with least relationship count

In [34]:
# Define the Cypher query
query = """
MATCH (n)-[r]->(m)
WITH n, m, COUNT(r) AS relationship_count
ORDER BY relationship_count ASC
LIMIT 1
RETURN n, m, relationship_count
"""

# Execute the query
cursor.execute(query)

# Fetch and display the results
result = cursor.fetchone()
if result:
    node1, node2, relationship_count = result
    print(f"Node 1: {node1}, Node 2: {node2}, Number of Relationships: {relationship_count}")
else:
    print("No nodes found.")


Node 1: None, Node 2: None, Number of Relationships: 0


In [4]:
def count_nodes(cursor):
    # Query to count the total number of nodes
    cursor.execute("MATCH (n) RETURN count(n)")
    # print(f"Total number of nodes in the database: {cursor.fetchone()[0]}")
    return cursor.fetchone()[0]

def count_relationships(cursor):
    # Query to count the total number of relationships
    cursor.execute("MATCH ()-[r]->() RETURN count(r)")
    total_relationships = cursor.fetchone()[0]
    # print(f"Total number of relationships in the database: {total_relationships}")
    return total_relationships

def show_storage_info():
    # Execute the SHOW STORAGE INFO commandytre
    cursor.execute("SHOW STORAGE INFO;")

    # Fetch and display the results
    results = cursor.fetchall()
    for result in results:
        print(result)


#### Build memgraph database -- with `MetaKG` Index

Build the `memgraph` db from the `Metakg` index.  
Should be run once -- similar to `admin.consolidate_metakg()`

First, lets view the data from the index

In [33]:
# # Delete all nodes and relationships
# cursor.execute("MATCH (n) DETACH DELETE n")

#### Build the Database

```
query = """
MERGE (s:Entity {name: $subject})
MERGE (o:Entity {name: $object})
MERGE (s)-[r:RELATED_TO {id: $unique_id}]->(o)
SET r.predicate = $predicate, r.api = $api_data
"""
```

In [5]:
index = ConsolidatedMetaKGDoc.Index.name

In [6]:
for edge in MetaKG.get_all_via_scan(size=1000, index=index):
    subject = edge['_source']['subject']
    object = edge['_source']['object']
    predicate = edge['_source']['predicate']
    api_data = edge['_source']['api']  # Additional data as properties

    # Construct a unique identifier for each relationship
    # This is just an example - modify it according to your data structure
    unique_id = f"{subject}-{predicate}-{object}"
    
    # Cypher query to create nodes with a static relationship
    query = """
    MERGE (s:Entity {name: $subject})
    MERGE (o:Entity {name: $object})
    MERGE (s)-[r:RELATED_TO {id: $unique_id}]->(o)
    SET r.predicate = $predicate, r.api = $api_data
    """
    params = {'subject': subject, 'object': object, 'unique_id': unique_id, 'predicate': predicate, 'api_data': api_data}
    cursor.execute(query, params)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


#### Inspect Database

In [11]:
show_storage_info()

('name', 'memgraph')
('vertex_count', 113)
('edge_count', 167827)
('average_degree', 2970.3893805309735)
('memory_usage', '680.99MiB')
('disk_usage', '611.04MiB')
('memory_allocated', '381.51MiB')
('allocation_limit', '20.00GiB')
('global_isolation_level', 'SNAPSHOT_ISOLATION')
('session_isolation_level', '')
('next_session_isolation_level', '')
('storage_mode', 'IN_MEMORY_TRANSACTIONAL')


In [6]:
count_nodes(cursor), count_relationships(cursor)

(0, 0)

View the most recently created nodes:

In [20]:
# Fetch the most recent nodes
query = "MATCH (n) RETURN n ORDER BY n.creationTime DESC LIMIT 5"
cursor.execute(query)

# Display the results
results = cursor.fetchall()
for node in results:
    print(node)


(<mgclient.Node(id=70, labels={'Entity'}, properties={'name': 'GeneticInheritance'}) at 0x110556670>,)
(<mgclient.Node(id=84, labels={'Entity'}, properties={'name': 'ActivityAndBehavior'}) at 0x110554d80>,)
(<mgclient.Node(id=83, labels={'Entity'}, properties={'name': 'BiologicalProcessOrActivity'}) at 0x110555230>,)
(<mgclient.Node(id=82, labels={'Entity'}, properties={'name': 'LifeStage'}) at 0x11072f7b0>,)
(<mgclient.Node(id=81, labels={'Entity'}, properties={'name': 'MacromolecularMachineMixin'}) at 0x11072dd70>,)


#### Run Queries 

Set sample query params

In [17]:
subject = 'Gene'
object = 'PhysiologicalProcess'

Query for a random document - This query matches all nodes (n) in the database, orders them randomly using the RAND() function, and limits the result to just one node.

In [17]:
query = "MATCH (n) RETURN n ORDER BY RAND() LIMIT 1"
cursor.execute(query)

# Fetch and display the result
result = cursor.fetchone()
if result:
    random_node = result[0]
    print(f"Random Node Properties: {random_node.properties}")
else:
    print("No nodes found in the database.")


Random Node Properties: {'name': 'ChemicalMixture'}


Return just 1 document - This query matches all nodes (n) in the database and limits the result to just one node.



In [16]:
query = "MATCH (n) RETURN n LIMIT 1"
cursor.execute(query)

# Fetch and display the result
result = cursor.fetchone()
if result:
    node = result[0]
    print(f"Node Properties: {node.properties}")
else:
    print("No nodes found in the database.")


Node Properties: {'name': 'SmallMolecule'}


Query that matches all relationships between nodes - 

In [15]:
query = """
MATCH (a:Entity {name: $subject})-[r]->(b:Entity {name: $object})
RETURN r
"""

cursor.execute(query, {'subject': subject, 'object': object})

# Fetch and display the results
results = cursor.fetchall()
for result in results:
    relationship = result[0]
    print(f"Relationship Properties: {relationship.properties}")


Relationship Properties: {'api': [{'bte': {'query_operation': {'input_separator': ',', 'method': 'post', 'params': None, 'path': '/query', 'path_params': None, 'request_body': None, 'server': 'https://automat.transltr.io/icees-kg/1.3', 'support_batch': True}}, 'name': 'Automat-icees-kg(Trapi v1.3.0)', 'smartapi': {'id': '44e7a1147ca8657f50af6bb25982762d', 'metadata': 'https://trapi-openapi.apps.renci.org/trapi/infores:automat-icees-kg/1.3.0', 'ui': 'https://smart-api.info/ui/44e7a1147ca8657f50af6bb25982762d'}, 'tags': ['translator', 'automat', 'trapi', 'bte-trapi'], 'x-translator': {'component': 'KP', 'team': ['Exposures Provider']}}, {'bte': {'query_operation': {'input_separator': ',', 'method': 'post', 'params': None, 'path': '/query', 'path_params': None, 'request_body': None, 'server': 'https://explanatory-agent.transltr.io/v1.3/', 'support_batch': True}}, 'name': 'Explanatory Agent API', 'smartapi': {'id': '95b8a4ac17ea779165a709185ac07f23', 'metadata': 'https://raw.githubusercont

Get the total count of the relationships

In [18]:
query = """
MATCH (a:Entity {name: $subject})-[r]->(b:Entity {name: $object})
RETURN COUNT(r) as relationship_count
"""

cursor.execute(query, {'subject': subject, 'object': object})

# Fetch and display the result
result = cursor.fetchone()
if result:
    print(f"Number of relationships between '{subject}' and '{object}': {result[0]}")
else:
    print("No relationships found.")


Number of relationships between 'Gene' and 'PhysiologicalProcess': 67


Print only the relation(predicate) name

In [19]:
list_query = """
MATCH (a:Entity {name: $subject})-[r]->(b:Entity {name: $object})
RETURN DISTINCT r.predicate as predicate
"""

# Execute the list query
cursor.execute(list_query, {'subject': subject, 'object': object})
list_results = cursor.fetchall()
pp.pprint(f"Predicates: {list_results}")



("Predicates: [('correlated_with',), ('affects',), ('affected_by',), "
 "('disrupts',), ('entity_positively_regulates_entity',), ('causes',), "
 "('genetic_association',), ('composed_primarily_of',), ('treated_by',), "
 "('gene_associated_with_condition',), ('associated_with_sensitivity_to',), "
 "('increases_response_to',), ('response_decreased_by',), "
 "('resistance_associated_with',), ('produced_by',), ('interacts_with',), "
 "('treats',), ('condition_associated_with_gene',), ('is_ameliorated_by',), "
 "('regulates',), ('response_increased_by',), "
 "('sensitivity_associated_with',), ('has_biomarker',), ('participates_in',), "
 "('decreased_likelihood_associated_with',), ('contributes_to',), "
 "('caused_by',), ('associated_with_increased_likelihood_of',), "
 "('has_participant',), ('entity_negatively_regulates_entity',), "
 "('decreases_response_to',), ('associated_with',), ('is_exacerbated_by',), "
 "('ameliorates',), ('positively_correlated_with',), ('disrupted_by',), "
 "('incr

In [53]:
query = """
MATCH (a:Entity {name: $subject})-[r]->(b:Entity {name: $object})
RETURN a.name, r.predicate, b.name
"""

cursor.execute(query, {'subject': subject, 'object': object})

# Fetch and display the results
results = cursor.fetchall()
for result in results:
    print(f"Subject: {result[0]}, Predicate: {result[1]}, Object: {result[2]}")


Subject: Gene, Predicate: correlated_with, Object: PhysiologicalProcess
Subject: Gene, Predicate: affects, Object: PhysiologicalProcess
Subject: Gene, Predicate: affected_by, Object: PhysiologicalProcess
Subject: Gene, Predicate: disrupts, Object: PhysiologicalProcess
Subject: Gene, Predicate: entity_positively_regulates_entity, Object: PhysiologicalProcess
Subject: Gene, Predicate: causes, Object: PhysiologicalProcess
Subject: Gene, Predicate: genetic_association, Object: PhysiologicalProcess
Subject: Gene, Predicate: composed_primarily_of, Object: PhysiologicalProcess
Subject: Gene, Predicate: treated_by, Object: PhysiologicalProcess
Subject: Gene, Predicate: gene_associated_with_condition, Object: PhysiologicalProcess
Subject: Gene, Predicate: associated_with_sensitivity_to, Object: PhysiologicalProcess
Subject: Gene, Predicate: increases_response_to, Object: PhysiologicalProcess
Subject: Gene, Predicate: response_decreased_by, Object: PhysiologicalProcess
Subject: Gene, Predicate: 

In [22]:
# Methods
def count_relationships_between_nodes(cursor, subject, object):
    query = """
    MATCH (a:Entity {name: $subject})-[r]->(b:Entity {name: $object})
    RETURN COUNT(r) as relationship_count
    """

    cursor.execute(query, {'subject': subject, 'object': object})

    # Fetch and display the result
    result = cursor.fetchone()
    if result:
        print(f"Number of relationships between '{subject}' and '{object}': {result[0]}")
    else:
        print("No relationships found.")


In [25]:
count_relationships_between_nodes(cursor, 'ChemicalMixture', 'SmallMolecule')

Number of relationships between 'ChemicalMixture' and 'SmallMolecule': 61


Get the paths

In [14]:
# this pair of nodes have 1 relationship between them, easy to test
subject = 'InformationResource'
object = 'Publication'


Setup Subgraph

In [26]:
query="""
MATCH p=(n:Entity {name: $start_name})-[r:RELATED_TO]->(m:Entity {name: $end_name})
WITH project(p) AS subgraph
RETURN subgraph;
"""

cursor.execute(query, {'start_name': subject, 'end_name': object})
results = cursor.fetchall()

In [27]:
results

[({'edges': [<mgclient.Relationship(start_id=22, end_id=3, type='RELATED_TO', properties={'api': [{'bte': {'query_operation': {'input_separator': ',', 'method': 'post', 'params': None, 'path': '/query', 'path_params': None, 'request_body': None, 'server': 'https://kg2.transltr.io/api/rtxkg2/v1.3', 'support_batch': True}}, 'name': 'RTX KG2 - TRAPI 1.3.0', 'smartapi': {'id': 'ccd4a8bb83de81401e9a27f1d8e7f948', 'metadata': 'https://raw.githubusercontent.com/RTXteam/RTX/production/code/UI/OpenAPI/python-flask-server/KG2/openapi_server/openapi/openapi.yaml', 'ui': 'https://smart-api.info/ui/ccd4a8bb83de81401e9a27f1d8e7f948'}, 'tags': ['meta_knowledge_graph', 'query', 'asyncquery', 'translator', 'trapi', 'entity', 'status', 'bte-trapi'], 'x-translator': {'component': 'KP', 'team': ['Expander Agent']}}, {'bte': {'query_operation': {'input_separator': ',', 'method': 'post', 'params': None, 'path': '/query', 'path_params': None, 'request_body': None, 'server': 'https://arax.ncats.io/api/rtxkg2/

In [16]:
# Define the Cypher query with QUERY MEMORY UNLIMITED
query = """
MATCH (start:Entity {name: $start_name}), (end:Entity {name: $end_name})
CALL algo.all_simple_paths(start, end, [], 3) YIELD path
RETURN path
"""
print("executing query...")
cursor.execute(query, {'start_name': subject, 'end_name': object})

# Fetch and display the results
results = cursor.fetchall()
for result in results:
    path = result[0]
    print(f"Path: {path}")

executing query...


mg_raw_transport_recv: connection closed by server


DatabaseError: failed to receive chunk size

Run `algo` methods to test package

In [18]:
# Define the Cypher query
query = """
MATCH (n:Label), (m:Label)
CALL nxalg.all_simple_paths(n, m, 5) YIELD paths
RETURN paths
"""

# Execute the query
cursor.execute(query)

# Fetch and display the results
results = cursor.fetchall()
for result in results:
    paths = result[0]
    print(f"Paths: {paths}")


---